# Workshop for Beginners

In [ ]:
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from tabulate import tabulate

import networkit as nk
from networkit import csbridge

NetworKit is organized in **modules**.

`nk.<module_name>.<feature_name>`

# 1. Reading a graph

NetworKit provides readers for popular graph formats:
- Edge lists
- KONECT: networks downloaded from http://konect.cc/networks/
- SNAP: networks downloaded from http://snap.stanford.edu/data/index.html
- More formats: https://networkit.github.io/dev-docs/python_api/graphio.html

Example: read a graph downloaded from KONECT.

We use the **graphio** module.

In [ ]:
# Create a graph reader
reader = nk.graphio.KONECTGraphReader()

# Read the graph
G = reader.read("./input/karate")

# Print some info about the graph
print(f"The graph has {G.numberOfNodes()} vertices and {G.numberOfEdges()} edges.")

In [ ]:
nk.csbridge.widget_from_graph(G)

# 2 Computation of Central Vertices

**Centrality:** identify important vertices in a graph.

## 2.1 Harmonic Closeness Centrality

Let $G = (V, E)$ be our input graph. The **harmonic closeness** of a vertex $v \in V$ is defined as:

$$
h(v) = \sum_{w \in V\setminus\{v\}}\frac{n - 1}{d(v, w)}
$$

- Measures how much a vertex $v$ is "close" to the other vertices
- Time complexity (per vertex): $\mathcal{O}(n + m)$ for unweighted graphs 
$\rightarrow$ for large graphs, use `TopHarmonicCloseness`

### Compute the harmonic closeness of all the vertices in $G$<br/>Visualize the distribution of the scores

In [ ]:
# General NetworKit algorithm pipeline

# 1. Create an intance of the algorithm
hc = nk.centrality.HarmonicCloseness(G, normalized=False)

# 2. Run the algorithm
hc.run()

# 3. Extract the results
harmonic = hc.scores()

# 4. Visualize the data
_ = plt.hist(harmonic)

In [ ]:
# Visualize harmonic centrality values
nk.csbridge.widget_from_graph(G, node_scores = harmonic)

In [ ]:
# Visualize with another color palette
palette = sns.color_palette("crest", G.numberOfNodes())
nk.csbridge.widget_from_graph(G, node_scores = harmonic, node_palette=palette)

## Top-$k$ Vertex Ranking

Often, we are interested in the ranking of the top-$k$ most central vertices.

In [ ]:
# Top-k algorithm to get the top-5 harmonic centrality ranking
topk_algo = nk.centrality.TopHarmonicCloseness(G, k = 5)

# Run the algorithm
topk_algo.run()

# Get the top-5 vertices and scores
topk_nodes, topk_scores = topk_algo.topkNodesList(), topk_algo.topkScoresList()

# Print the top-5 ranking
print(tabulate([(u, s) for u, s in zip(topk_nodes, topk_scores)], headers = ["Vertex", "Score"]))

In [ ]:
# Label vertices:
# True => is in top-5
# False => is not in top-5

in_top_k = [False] * G.numberOfNodes()

for u in topk_nodes:
    in_top_k[u] = True
    
nk.csbridge.widget_from_graph(G, node_partition = nk.structures.Partition(G.numberOfNodes(), in_top_k))

# 3. Graph Generators

The **generators** module provides a wide range of models to generate graphs.

` G = nk.generators.<generator_constructor>.generate()`

### 3.1 Random Hyperbolic Generator

Generates **complex networks** with a power-law degree distribution. Parameters:

`n` number of vertices in the graph

`avg_deg` average vertex degree

`gamma` exponent of the power-law degree distribution

In [ ]:
# Parameters:
n, avg_deg, exp = 1e5, 10, 3

# Gererate the graph
G_cplx = nk.generators.HyperbolicGenerator(n, avg_deg, exp).generate()

# Degree of every vertex
degrees = [G_cplx.degree(u) for u in G_cplx.iterNodes()]

# Sort by degree (descending order)
deg_count = np.array(sorted(Counter(degrees).items(), key=lambda x: x[0]))

# Plot degree distribution (log-log plot)
fig, ax = plt.subplots()
ax.set_xscale('log', base=2)
ax.set_yscale('log', base=2)
ax.set_xlabel("Degree")
ax.set_ylabel("#of vertices")
ax.loglog(deg_count[1:, 0], deg_count[1:, 1], label='Degree distribution')
_ = ax.legend()

### 3.2 Clustered Random Graph Generator
Generates random graphs **with clusters**.

`n`: number of vertices in the graph

`num_clusters`: number of clusters/communities in the graph

`p_in`: intra-cluster edge probability

`p_out`: inter-cluster edge probability

In [ ]:
nk.engineering.setSeed(42, False)

# Parameters
n, num_clusters, p_in, p_out = 60, 3, 0.3, 0.005
cluster_gen = nk.generators.ClusteredRandomGraphGenerator(n, num_clusters, p_in, p_out)

# Generate a clustered random graph
G_cluster = cluster_gen.generate()

# Visualize the graph and its communities
nk.csbridge.widget_from_graph(G_cluster, node_partition = cluster_gen.getCommunities())

## 4. Community Detection

### Objective
Identify the community structure of a graph.

### Methodology: Modularity Optimization
Maximize the modularity:

$$
Q = \frac{1}{2M}\sum_{i \in V}\sum_{j \in V}\left( w(i, j) - \frac{\deg(i)\deg(j)}{2M} \right)
\delta\left(c_i, c_j\right)
$$

### Popular algorithm: Louvain Method

In [ ]:
# 1. Create an instance of the Parallel Louvain Method
plm = nk.community.PLM(G_cluster)

# 2. Run the algorithm
plm.run()

# 3. Draw the partitioning of the communities
nk.csbridge.widget_from_graph(G_cluster, node_partition = plm.getPartition())